In [3]:
import os
import numpy as np
import pandas as pd
import sklearn as skl
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import (
    RidgeCV,
    MultiTaskLassoCV,
    MultiTaskElasticNetCV,
    LogisticRegressionCV,
)
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor, MultiOutputClassifier
from tqdm import tqdm
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

In [15]:
from xgboost import XGBClassifier

In [8]:
datapath = os.path.join("data", "US", "us_data_weekly.csv")
dus_weekly = pd.read_csv(datapath, index_col=0)
dus_weekly = dus_weekly.dropna()

In [9]:
def add_ts_features(df, cols, max_lag=20, windows=[10, 20, 50]):
    all_features = {}

    for col in cols:
        y = df[col]

        # --- Rolling statistics ---
        for w in windows:
            roll = y.rolling(w)
            all_features[f"{col}_mean_{w}"] = roll.mean()
            all_features[f"{col}_std_{w}"] = roll.std()
            all_features[f"{col}_q25_{w}"] = roll.quantile(0.25)
            all_features[f"{col}_q75_{w}"] = roll.quantile(0.75)
            all_features[f"{col}_q05_{w}"] = roll.quantile(0.05)
            all_features[f"{col}_q90_{w}"] = roll.quantile(0.9)
            all_features[f"{col}_range_{w}"] = roll.max() - roll.min()

            # Z-score et momentum
            all_features[f"{col}_zscore_{w}"] = (y - roll.mean()) / roll.std()
            all_features[f"{col}_momentum_{w}"] = y - y.shift(w)

        # Ratio de moyennes rapides / lentes
        all_features[f"{col}_ratio_{10}_{50}"] = (
            y.rolling(10).mean() / y.rolling(50).mean()
        )

        # Volatilité annualisée approx
        all_features[f"{col}_vol_20"] = y.rolling(20).std() * np.sqrt(52)

        # --- Lags bruts ---
        for lag in [1, 2, 3, 4, 5, 10, 15, 20, 25, 30, 40, 50]:
            all_features[f"{col}_lag_{lag}"] = y.shift(lag - 1)

        # --- Autocorrélations (in-sample) ---

        acf_vals = acf(y.dropna(), nlags=max_lag, fft=True)
        for lag in range(1, max_lag + 1):
            all_features[f"{col}_autocorr_{lag}"] = acf_vals[lag]

    # --- Construction finale ---
    features = pd.DataFrame(all_features, index=df.index)
    return features


In [10]:
cols = [
    "DGS1MO",
    "DGS3MO",
    "DGS6MO",
    "DGS1",
    "DGS2",
    "DGS3",
    "DGS5",
    "DGS7",
    "DGS10",
    "DGS20",
    "DGS30",
]

dus_features = add_ts_features(dus_weekly, cols)
dus_features = dus_features.dropna()
dus_weekly = dus_weekly.merge(
    dus_features, how="inner", left_index=True, right_index=True
)

In [11]:
for col in cols:
    dus_weekly[f"Y_{col}"] = (dus_weekly[col] > 0).astype(int).shift(-1)


dus_weekly = dus_weekly.dropna()
dus_weekly = dus_weekly.replace([np.inf, -np.inf], np.nan)
dus_weekly = dus_weekly.ffill()

# we can now remove the original yield columns
dus_weekly = dus_weekly.drop(columns=cols)

In [12]:
Yw = dus_weekly[[f"Y_{col}" for col in cols]]
Xw = dus_weekly.drop(columns=[f"Y_{col}" for col in cols])

print(f"there are {Xw.shape[1]} features in the dataset")

there are 738 features in the dataset


We augmented the dataset by adding a very large amount of features created from the yields time series. however, many of these features are not informative so we need to remove them before training our model. 

We'll filter features by keeping only those with a mutual information score above than a given threshold. We'll create several datasets of features containing features filtered for the thresholds 0.03, 0.035, 0.04, 0.05 and train models on these specific datasets. 

In [13]:
threshold_mi = 0.03

initial_number_of_features = Xw.shape[1]

selected_features_w = set()
for col in tqdm(Yw.columns):
    mi = mutual_info_classif(Xw, Yw[col])
    top_features_i = Xw.columns[mi > threshold_mi]
    selected_features_w.update(top_features_i)

print(
    f"we removed {initial_number_of_features - len(selected_features_w)} features in the weekly dataset for threshold {threshold_mi}"
)
Xw = Xw[list(selected_features_w)]


threshold_mi = 0.035

selected_features_w = set()
for col in tqdm(Yw.columns):
    mi = mutual_info_classif(Xw, Yw[col])
    top_features_i = Xw.columns[mi > threshold_mi]
    selected_features_w.update(top_features_i)

print(
    f"we removed {initial_number_of_features - len(selected_features_w)} features in the weekly dataset for threshold {threshold_mi}"
)
Xw35 = Xw[list(selected_features_w)]

threshold_mi = 0.04

selected_features_w = set()
for col in tqdm(Yw.columns):
    mi = mutual_info_classif(Xw, Yw[col])
    top_features_i = Xw.columns[mi > threshold_mi]
    selected_features_w.update(top_features_i)

print(
    f"we removed {initial_number_of_features - len(selected_features_w)} features in the weekly dataset for threshold {threshold_mi}"
)
Xw4 = Xw[list(selected_features_w)]


threshold_mi = 0.05

selected_features_w = set()
for col in tqdm(Yw.columns):
    mi = mutual_info_classif(Xw, Yw[col])
    top_features_i = Xw.columns[mi > threshold_mi]
    selected_features_w.update(top_features_i)

print(
    f"we removed {initial_number_of_features - len(selected_features_w)} features in the weekly dataset for threshold {threshold_mi}"
)
Xw5 = Xw[list(selected_features_w)]

100%|██████████| 11/11 [00:09<00:00,  1.20it/s]


we removed 476 features in the weekly dataset for threshold 0.03


100%|██████████| 11/11 [00:03<00:00,  3.39it/s]


we removed 614 features in the weekly dataset for threshold 0.035


100%|██████████| 11/11 [00:03<00:00,  3.41it/s]


we removed 645 features in the weekly dataset for threshold 0.04


100%|██████████| 11/11 [00:03<00:00,  3.40it/s]

we removed 705 features in the weekly dataset for threshold 0.05


Notice that our macro and market variables do not share much mutual information with the target since most of these variables are removed with the threshold we used:


In [ ]:
print("Number of variables in the dataset with MI >0.03:", Xw.shape[1])
print(
    "Macro and market variables in the dataset with MI >0.03:",
    [col for col in Xw.columns if "DGS" not in col],
)
print("Number of variables in the dataset with MI >0.035:", Xw35.shape[1])
print(
    "Macro and market variables in the dataset with MI >0.035:",
    [col for col in Xw35.columns if "DGS" not in col],
)
print("Number of variables in the dataset with MI >0.04:", Xw4.shape[1])
print(
    "Macro and market variables in the dataset with MI >0.04:",
    [col for col in Xw4.columns if "DGS" not in col],
)
print("Number of variables in the dataset with MI >0.05:", Xw5.shape[1])
print(
    "Macro and market variables in the dataset with MI >0.05:",
    [col for col in Xw4.columns if "DGS" not in col],
)

Number of variables in the dataset with MI >0.03: 262
Macro and market variables in the dataset with MI >0.03: ['USFIRE', 'USCONS', 'CUUR0000SA0L2', 'HOUST', 'CUSR0000SAC', 'IRLTLT01GBM156N', 'IRLTLT01AUM156N', 'DTCTHFNM', 'GFDEGDQ188S', 'USTPU', 'GFDEBTN', 'CPIAUCSL', 'FEDFUNDS', 'SRVPRD', 'IRLTLT01CAM156N', 'MANEMP', 'IRLTLT01DEM156N', 'PAYEMS']
Number of variables in the dataset with MI >0.035: 124
Macro and market variables in the dataset with MI >0.035: ['IRLTLT01AUM156N', 'GFDEGDQ188S', 'USTPU', 'IRLTLT01DEM156N']
Number of variables in the dataset with MI >0.04: 93
Macro and market variables in the dataset with MI >0.04: ['USCONS', 'DTCTHFNM', 'USTPU', 'IRLTLT01CAM156N']
Number of variables in the dataset with MI >0.05: 33
Macro and market variables in the dataset with MI >0.05: ['USCONS', 'DTCTHFNM', 'USTPU', 'IRLTLT01CAM156N']


## C) XGBoost

We'll now train XGboost models to see whether it can capture non linearities in the data and improve accuracy of out-of-sample predictions. Once again, we'll train some models on the 4 datasets that we created. 

For the first dataset (with features with a MI>0.03), we'll also train a model that performs a PCA to reduce the number of features before applying XGboost. 

The pipelines that we'll use are the following:

In [16]:
window_train = 52 * 15
window_pred = 4
tscv = TimeSeriesSplit(n_splits=4)


# parameters to test in cross validation
param_grid = {
    "xgb__estimator__n_estimators": [100, 200],
    "xgb__estimator__learning_rate": [0.01, 0.05],
    "xgb__estimator__max_depth": [4, 7],
    "xgb__estimator__subsample": [0.5, 0.7],
    "xgb__estimator__colsample_bytree": [0.4, 0.8],
    "xgb__estimator__min_child_weight": [5, 10],
    "xgb__estimator__reg_alpha": [0.5, 1.0],
    "xgb__estimator__reg_lambda": [5, 10],
}


# pipeline without PCA
pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        (
            "xgb",
            MultiOutputClassifier(
                XGBClassifier(
                    objective="binary:logistic",
                    use_label_encoder=False,
                    n_jobs=-1,
                    random_state=42,
                    verbosity=0,
                )
            ),
        ),
    ]
)


# pipeline with PCA
pipepca = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=0.99)),
        (
            "xgb",
            MultiOutputClassifier(
                XGBClassifier(
                    objective="binary:logistic",
                    use_label_encoder=False,
                    n_jobs=-1,
                    random_state=42,
                    verbosity=0,
                )
            ),
        ),
    ]
)


# and we'll use gridsearchCV to cross validate the model:
GridSearchCV(
    pipepca,  # or pipe
    param_grid,
    cv=tscv,
    scoring="accuracy",
    n_jobs=-1,
    verbose=0,
)


,estimator,"Pipeline(step...None, ...)))])"
,param_grid,"{'xgb__estimator__colsample_bytree': [0.4, 0.8], 'xgb__estimator__learning_rate': [0.01, 0.05], 'xgb__estimator__max_depth': [4, 7], 'xgb__estimator__min_child_weight': [5, 10], ...}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,TimeSeriesSpl...est_size=None)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,copy,True


We now train XGBoost models for the dataset containing features with a mutual information above 0.03:

In [17]:
accuracies = {col: [] for col in Yw.columns}
params_by_target = {col: [] for col in Yw.columns}
feature_importance = {col: [] for col in Yw.columns}
y_true = []
y_pred = []

for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):
    end_train = start + window_train
    end_pred = end_train + window_pred

    Xw_train = Xw.iloc[start:end_train]
    Yw_train = Yw.iloc[start:end_train]
    Xw_test = Xw.iloc[end_train:end_pred]
    Yw_test = Yw.iloc[end_train:end_pred]

    y_true.append(Yw_test)

    grid = GridSearchCV(
        pipe, param_grid, cv=tscv, scoring="accuracy", n_jobs=-1, verbose=0
    )

    grid.fit(Xw_train, Yw_train)
    best_model = grid.best_estimator_

    Yw_pred = best_model.predict(Xw_test)
    y_pred.append(Yw_pred)

    for i, col in enumerate(Yw_train.columns):
        acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
        accuracies[col].append(acc)
        print(col, np.mean(accuracies[col]))

        # feature importance pour la i-ème sortie
        est = best_model.named_steps["xgb"].estimators_[i]  # XGBClassifier
        importance = est.feature_importances_
        feature_importance[col].append(importance)

        # hyperparamètres de l'estimateur i
        params = est.get_params()
        params_by_target[col].append(params)


acc = pd.DataFrame(accuracies, columns=Yw.columns)
params = pd.DataFrame(params_by_target, columns=Yw.columns)
feature_imp = pd.DataFrame(feature_importance, columns=Yw.columns)
Y_true_flat = np.array(y_true).reshape(
    -1, np.array(y_true).shape[2]
)  # aplati les 2 premières dimensions en une seule
ytrue = pd.DataFrame(
    Y_true_flat, index=Yw.iloc[780 : Yw.shape[0] - 1].index, columns=Yw.columns
)
Y_pred_flat = np.array(y_pred).reshape(
    -1, np.array(y_pred).shape[2]
)  # aplati les 2 premières dimensions en une seule
ypred = pd.DataFrame(
    Y_pred_flat, index=Yw.iloc[780 : Yw.shape[0] - 1].index, columns=Yw.columns
)


acc.to_csv("accuracies xgboost, 15y train test.csv")
params.to_csv("params xgboost, 15y train test.csv")
feature_imp.to_csv("feature importance xgboost, 15y train test.csv")
ypred.to_csv("forecast xgboost, 15y train test.csv")
ytrue.to_csv("true values xgboost, 15y train test.csv")

print(acc.mean())


  1%|▏         | 1/79 [02:57<3:50:16, 177.14s/it]

Y_DGS1MO 0.75
Y_DGS3MO 0.75
Y_DGS6MO 0.5
Y_DGS1 0.0
Y_DGS2 0.0
Y_DGS3 0.25
Y_DGS5 0.25
Y_DGS7 0.25
Y_DGS10 0.5
Y_DGS20 0.5
Y_DGS30 1.0


  1%|▏         | 1/79 [03:51<5:01:07, 231.64s/it]


KeyboardInterrupt: 

We now train XGBoost models for the dataset containing features with a mutual information above 0.03, but this time we include a PCA to reduce the number of features. 

In [ ]:
accuracies = {col: [] for col in Yw.columns}
params_by_target = {col: [] for col in Yw.columns}
feature_importance = {col: [] for col in Yw.columns}
y_true = []
y_pred = []

for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):
    end_train = start + window_train
    end_pred = end_train + window_pred

    Xw_train = Xw.iloc[start:end_train]
    Yw_train = Yw.iloc[start:end_train]
    Xw_test = Xw.iloc[end_train:end_pred]
    Yw_test = Yw.iloc[end_train:end_pred]

    y_true.append(Yw_test)

    grid = GridSearchCV(pipepca,
                        param_grid,
                        cv=tscv,
                        scoring='accuracy',
                        n_jobs=-1,
                        verbose=0)

    grid.fit(Xw_train, Yw_train)
    best_model = grid.best_estimator_

    Yw_pred = best_model.predict(Xw_test)
    y_pred.append(Yw_pred)

    for i, col in enumerate(Yw_train.columns):
        acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
        accuracies[col].append(acc)
        print(col, np.mean(accuracies[col]))

        # feature importance pour la i-ème sortie
        est = best_model.named_steps['xgb'].estimators_[i]   # XGBClassifier
        importance = est.feature_importances_
        feature_importance[col].append(importance)

        # hyperparamètres de l'estimateur i
        params = est.get_params()
        params_by_target[col].append(params)



acc = pd.DataFrame(accuracies,columns = Yw.columns)
params = pd.DataFrame(params_by_target, columns = Yw.columns) 
feature_imp = pd.DataFrame(feature_importance, columns = Yw.columns)
Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)
Y_pred_flat = np.array(y_pred).reshape(-1, np.array(y_pred).shape[2])  # aplati les 2 premières dimensions en une seule
ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)


acc.to_csv('accuracies xgboost, pca, 15y train test.csv')
params.to_csv('params xgboost, pca, 15y train test.csv')
feature_imp.to_csv('feature importance xgboost, pca, 15y train test.csv')
ypred.to_csv('forecast xgboost, pca, 15y train test.csv')
ytrue.to_csv('true values xgboost, pca, 15y train test.csv')

print(acc.mean())


Results are surprisingly very disappointing for XGBoost. When looking at predictions made by this model, we see that the prediction vectors are almost always 0. There are very few ones in the predictions, and since classes are well balanced, this results in a performance close to predicting yields randomly. 

We will now try to run this model on the other datasets of features such that their mutual information is >0.035 or 0.04 or 0.05 to see if there are any improvements.

For the dataset with MI >0.035:

In [ ]:
accuracies = {col: [] for col in Yw.columns}
params_by_target = {col: [] for col in Yw.columns}
feature_importance = {col: [] for col in Yw.columns}
y_true = []
y_pred = []

for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):
    end_train = start + window_train
    end_pred = end_train + window_pred

    Xw_train = Xw35.iloc[start:end_train]
    Yw_train = Yw.iloc[start:end_train]
    Xw_test = Xw35.iloc[end_train:end_pred]
    Yw_test = Yw.iloc[end_train:end_pred]

    y_true.append(Yw_test)

    grid = GridSearchCV(pipe,
                        param_grid,
                        cv=tscv,
                        scoring='accuracy',
                        n_jobs=-1,
                        verbose=0)

    grid.fit(Xw_train, Yw_train)
    best_model = grid.best_estimator_

    Yw_pred = best_model.predict(Xw_test)
    y_pred.append(Yw_pred)

    for i, col in enumerate(Yw_train.columns):
        acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
        accuracies[col].append(acc)
        print(col, np.mean(accuracies[col]))

        # feature importance pour la i-ème sortie
        est = best_model.named_steps['xgb'].estimators_[i]   # XGBClassifier
        importance = est.feature_importances_
        feature_importance[col].append(importance)

        # hyperparamètres de l'estimateur i
        params = est.get_params()
        params_by_target[col].append(params)



acc = pd.DataFrame(accuracies,columns = Yw.columns)
params = pd.DataFrame(params_by_target, columns = Yw.columns) 
feature_imp = pd.DataFrame(feature_importance, columns = Yw.columns)
Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)
Y_pred_flat = np.array(y_pred).reshape(-1, np.array(y_pred).shape[2])  # aplati les 2 premières dimensions en une seule
ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)

acc.to_csv('accuracies xgboost, MI 0.035, 15y train test.csv')
params.to_csv('params xgboost, MI 0.035, 15y train test.csv')
feature_imp.to_csv('feature importance xgboost, MI 0.035, 15y train test.csv')
ypred.to_csv('forecast xgboost, MI 0.035, 15y train test.csv')
ytrue.to_csv('true values xgboost, MI 0.035, 15y train test.csv')


For the dataset with MI > 0.04:

In [ ]:
accuracies = {col: [] for col in Yw.columns}
params_by_target = {col: [] for col in Yw.columns}
feature_importance = {col: [] for col in Yw.columns}
y_true = []
y_pred = []

for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):
    end_train = start + window_train
    end_pred = end_train + window_pred

    Xw_train = Xw4.iloc[start:end_train]
    Yw_train = Yw.iloc[start:end_train]
    Xw_test = Xw4.iloc[end_train:end_pred]
    Yw_test = Yw.iloc[end_train:end_pred]

    y_true.append(Yw_test)

    grid = GridSearchCV(pipe,
                        param_grid,
                        cv=tscv,
                        scoring='accuracy',
                        n_jobs=-1,
                        verbose=0)

    grid.fit(Xw_train, Yw_train)
    best_model = grid.best_estimator_

    Yw_pred = best_model.predict(Xw_test)
    y_pred.append(Yw_pred)

    for i, col in enumerate(Yw_train.columns):
        acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
        accuracies[col].append(acc)
        print(col, np.mean(accuracies[col]))

        # feature importance pour la i-ème sortie
        est = best_model.named_steps['xgb'].estimators_[i]   # XGBClassifier
        importance = est.feature_importances_
        feature_importance[col].append(importance)

        # hyperparamètres de l'estimateur i
        params = est.get_params()
        params_by_target[col].append(params)



acc = pd.DataFrame(accuracies,columns = Yw.columns)
params = pd.DataFrame(params_by_target, columns = Yw.columns) 
feature_imp = pd.DataFrame(feature_importance, columns = Yw.columns)
Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)
Y_pred_flat = np.array(y_pred).reshape(-1, np.array(y_pred).shape[2])  # aplati les 2 premières dimensions en une seule
ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)

acc.to_csv('accuracies xgboost, MI 0.04, 15y train test.csv')
params.to_csv('params xgboost, MI 0.04, 15y train test.csv')
feature_imp.to_csv('feature importance xgboost, MI 0.04, 15y train test.csv')
ypred.to_csv('forecast xgboost, MI 0.04, 15y train test.csv')
ytrue.to_csv('true values xgboost, MI 0.04, 15y train test.csv')

  1%|▏         | 1/79 [10:36<13:47:11, 636.30s/it]

Y_DGS1MO 0.5
Y_DGS3MO 0.75
Y_DGS6MO 0.5
Y_DGS1 0.0
Y_DGS2 0.0
Y_DGS3 0.25
Y_DGS5 0.25
Y_DGS7 0.0
Y_DGS10 0.5
Y_DGS20 0.5
Y_DGS30 0.75


  3%|▎         | 2/79 [20:35<13:08:54, 614.74s/it]

Y_DGS1MO 0.5
Y_DGS3MO 0.625
Y_DGS6MO 0.5
Y_DGS1 0.125
Y_DGS2 0.125
Y_DGS3 0.25
Y_DGS5 0.25
Y_DGS7 0.125
Y_DGS10 0.375
Y_DGS20 0.5
Y_DGS30 0.625


  4%|▍         | 3/79 [29:53<12:25:26, 588.51s/it]

...

 99%|█████████▊| 78/79 [10:32:58<06:10, 370.23s/it]

Y_DGS1MO 0.6121794871794872
Y_DGS3MO 0.6826923076923077
Y_DGS6MO 0.6698717948717948
Y_DGS1 0.5192307692307693
Y_DGS2 0.5224358974358975
Y_DGS3 0.4967948717948718
Y_DGS5 0.5
Y_DGS7 0.5
Y_DGS10 0.47115384615384615
Y_DGS20 0.5032051282051282
Y_DGS30 0.5


100%|██████████| 79/79 [10:39:00<00:00, 485.32s/it]

Y_DGS1MO 0.6139240506329114
Y_DGS3MO 0.6867088607594937
Y_DGS6MO 0.6740506329113924
Y_DGS1 0.5189873417721519
Y_DGS2 0.5189873417721519
Y_DGS3 0.4936708860759494
Y_DGS5 0.49683544303797467
Y_DGS7 0.49683544303797467
Y_DGS10 0.46835443037974683
Y_DGS20 0.5
Y_DGS30 0.49683544303797467


Y_DGS1MO    0.613924
Y_DGS3MO    0.686709
Y_DGS6MO    0.674051
Y_DGS1      0.518987
Y_DGS2      0.518987
Y_DGS3      0.493671
Y_DGS5      0.496835
Y_DGS7      0.496835
Y_DGS10     0.468354
Y_DGS20     0.500000
Y_DGS30     0.496835
dtype: float64


And for the dataset with MI > 0.05:

In [ ]:
accuracies = {col: [] for col in Yw.columns}
params_by_target = {col: [] for col in Yw.columns}
feature_importance = {col: [] for col in Yw.columns}
y_true = []
y_pred = []

for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):
    end_train = start + window_train
    end_pred = end_train + window_pred

    Xw_train = Xw5.iloc[start:end_train]
    Yw_train = Yw.iloc[start:end_train]
    Xw_test = Xw5.iloc[end_train:end_pred]
    Yw_test = Yw.iloc[end_train:end_pred]

    y_true.append(Yw_test)

    grid = GridSearchCV(pipe,
                        param_grid,
                        cv=tscv,
                        scoring='accuracy',
                        n_jobs=-1,
                        verbose=0)

    grid.fit(Xw_train, Yw_train)
    best_model = grid.best_estimator_

    Yw_pred = best_model.predict(Xw_test)
    y_pred.append(Yw_pred)

    for i, col in enumerate(Yw_train.columns):
        acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
        accuracies[col].append(acc)
        print(col, np.mean(accuracies[col]))

        # feature importance pour la i-ème sortie
        est = best_model.named_steps['xgb'].estimators_[i]   # XGBClassifier
        importance = est.feature_importances_
        feature_importance[col].append(importance)

        # hyperparamètres de l'estimateur i
        params = est.get_params()
        params_by_target[col].append(params)



acc = pd.DataFrame(accuracies,columns = Yw.columns)
params = pd.DataFrame(params_by_target, columns = Yw.columns) 
feature_imp = pd.DataFrame(feature_importance, columns = Yw.columns)


Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)


Y_pred_flat = np.array(y_pred).reshape(-1, np.array(y_pred).shape[2])  # aplati les 2 premières dimensions en une seule
ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)



acc.to_csv('accuracies xgboost, MI 0.05, 15y train test.csv')
params.to_csv('params xgboost, MI 0.05, 15y train test.csv')
feature_imp.to_csv('feature importance xgboost, MI 0.05, 15y train test.csv')
ypred.to_csv('forecast xgboost, MI 0.05, 15y train test.csv')
ytrue.to_csv('true values xgboost, MI 0.05, 15y train test.csv')

print(acc.mean())


  1%|▏         | 1/79 [04:08<5:23:14, 248.64s/it]

Y_DGS1MO 0.75
Y_DGS3MO 0.75
Y_DGS6MO 0.5
Y_DGS1 0.5
Y_DGS2 0.25
Y_DGS3 0.5
Y_DGS5 0.75
Y_DGS7 0.75
Y_DGS10 0.5
Y_DGS20 0.75
Y_DGS30 0.75


  3%|▎         | 2/79 [07:38<4:49:39, 225.71s/it]

Y_DGS1MO 0.625
Y_DGS3MO 0.625
Y_DGS6MO 0.375
Y_DGS1 0.5
Y_DGS2 0.25
Y_DGS3 0.5
Y_DGS5 0.625
Y_DGS7 0.625
Y_DGS10 0.5
Y_DGS20 0.625
Y_DGS30 0.625


  4%|▍         | 3/79 [11:00<4:32:13, 214.91s/it]

...

 99%|█████████▊| 78/79 [4:13:18<03:03, 183.13s/it]

Y_DGS1MO 0.6185897435897436
Y_DGS3MO 0.6730769230769231
Y_DGS6MO 0.657051282051282
Y_DGS1 0.5576923076923077
Y_DGS2 0.5352564102564102
Y_DGS3 0.5032051282051282
Y_DGS5 0.4935897435897436
Y_DGS7 0.5096153846153846
Y_DGS10 0.5
Y_DGS20 0.5160256410256411
Y_DGS30 0.5064102564102564


100%|██████████| 79/79 [4:16:19<00:00, 194.67s/it]

Y_DGS1MO 0.620253164556962
Y_DGS3MO 0.6772151898734177
Y_DGS6MO 0.6550632911392406
Y_DGS1 0.5569620253164557
Y_DGS2 0.5316455696202531
Y_DGS3 0.5
Y_DGS5 0.49050632911392406
Y_DGS7 0.5063291139240507
Y_DGS10 0.49683544303797467
Y_DGS20 0.5126582278481012
Y_DGS30 0.5031645569620253


Y_DGS1MO    0.620253
Y_DGS3MO    0.677215
Y_DGS6MO    0.655063
Y_DGS1      0.556962
Y_DGS2      0.531646
Y_DGS3      0.500000
Y_DGS5      0.490506
Y_DGS7      0.506329
Y_DGS10     0.496835
Y_DGS20     0.512658
Y_DGS30     0.503165
dtype: float64
